In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


df = pd.read_csv('ab_experiment_data.csv')
df

,user_id,exp_group,order_id,order_sum,profit_from_order
0,2,control,1,2093.470956,418.694191
1,2,control,2,2043.904347,408.780869
2,2,control,3,1917.627651,383.525530
3,2,control,4,1848.305865,369.661173
4,4,control,5,2158.413140,431.682628
...,...,...,...,...,...
304531,99999,test,304532,1781.432336,356.286467
304532,99999,test,304533,1868.117987,373.623597
304533,99999,test,304534,2133.494374,426.698875
304534,99999,test,304535,1999.441785,399.888357


In [14]:
alpha = 0.05

# Для каждой пары (user_id, exp_group) вычисляем:
#   - n_orders: число уникальных заказов
#   - total_revenue: сумма всех order_sum этого пользователя
#   - total_profit: сумма всех profit_from_order этого пользователя
user_metrics = df.groupby(['user_id', 'exp_group']).agg(
    n_orders=('order_id', 'nunique'),
    total_revenue=('order_sum', 'sum'),
    total_profit=('profit_from_order', 'sum')
).reset_index()

# Добавляем средний чек на уровне пользователя
user_metrics['average_check'] = user_metrics['total_revenue'] / user_metrics['n_orders']

# Разбиваем пользователей на группы
group_A = user_metrics[user_metrics['exp_group'] == 'control']
group_B = user_metrics[user_metrics['exp_group'] == 'test']

# Строим агрегаты по группам
summary = {
    'group': ['A', 'B'],
    'n_users': [group_A['user_id'].nunique(), group_B['user_id'].nunique()],
    'sum_orders': [group_A['n_orders'].sum(),     group_B['n_orders'].sum()],
    'sum_revenue': [group_A['total_revenue'].sum(), group_B['total_revenue'].sum()],
    'sum_profit': [group_A['total_profit'].sum(),   group_B['total_profit'].sum()],
    'mean_orders_per_user': [group_A['n_orders'].mean(), group_B['n_orders'].mean()],
    'mean_revenue_per_user': [group_A['total_revenue'].mean(), group_B['total_revenue'].mean()],
    'mean_profit_per_user': [group_A['total_profit'].mean(),    group_B['total_profit'].mean()],
    'mean_check_per_order_group': [
        (group_A['total_revenue'].sum() / group_A['n_orders'].sum())
        if group_A['n_orders'].sum() > 0 else np.nan,
        (group_B['total_revenue'].sum() / group_B['n_orders'].sum())
        if group_B['n_orders'].sum() > 0 else np.nan
    ]
}

summary_df = pd.DataFrame(summary)
print("=== Сводка по группам A и B (агрегаты) ===")
print(summary_df.to_string(index=False))

# Двухвыборочные t-тесты для каждой метрики на уровне пользователя
# a) n_orders
t_orders,   p_orders   = stats.ttest_ind(group_B['n_orders'],       group_A['n_orders'],       equal_var=False)
# b) total_revenue
t_revenue,  p_revenue  = stats.ttest_ind(group_B['total_revenue'],  group_A['total_revenue'],  equal_var=False)
# c) average_check
t_check,    p_check    = stats.ttest_ind(group_B['average_check'],  group_A['average_check'],  equal_var=False)
# d) total_profit
t_profit,   p_profit   = stats.ttest_ind(group_B['total_profit'],   group_A['total_profit'],   equal_var=False)

results = {
    'metric': ['n_orders', 'total_revenue', 'average_check', 'total_profit'],
    't_statistic': [t_orders,  t_revenue,  t_check,  t_profit],
    'p_value':     [p_orders,  p_revenue,  p_check,  p_profit]
}
results_df = pd.DataFrame(results)
print("\n=== Результаты t-тестов ===")
print(results_df.to_string(index=False))

# Инкрементальная прибыль за один дополнительный заказ
diff_orders = summary_df.loc[1, 'sum_orders'] - summary_df.loc[0, 'sum_orders']
diff_profit = summary_df.loc[1, 'sum_profit'] - summary_df.loc[0, 'sum_profit']
if diff_orders != 0:
    incremental_profit_per_order = diff_profit / diff_orders
else:
    incremental_profit_per_order = np.nan

print(f"\nРазница в количестве заказов (B - A): {diff_orders}")
print(f"Разница в суммарной прибыли  (B - A): {diff_profit:.2f} руб.")
print(f"Инкрементальная прибыль за 1 дополнительный заказ: {incremental_profit_per_order:.2f} руб.\n")

print("=== Выводы по вопросам (a)–(e) ===\n")

# (a) Удалось ли вырастить количество заказов между группами?
sum_orders_A = summary_df.loc[0, 'sum_orders']
sum_orders_B = summary_df.loc[1, 'sum_orders']
print("(a) Количество заказов:")
print(f"    - Группа A: {sum_orders_A}, Группа B: {sum_orders_B}.")
if sum_orders_B > sum_orders_A:
    direction_orders = "увеличилось"
else:
    direction_orders = "не увеличилось"
stat_label_orders = "статистически значимо" if (p_orders < alpha) else "не статистически значимо"
print(f"    - Количество заказов в группе B {direction_orders} по сравнению с A (p = {p_orders:.4f}, {stat_label_orders}).\n")

# (b) Как повлияли на общую сумму, которую пользователи оплатили за заказы?
sum_revenue_A = summary_df.loc[0, 'sum_revenue']
sum_revenue_B = summary_df.loc[1, 'sum_revenue']
print("(b) Суммарная выручка:")
print(f"    - Группа A: {sum_revenue_A:.2f} руб, Группа B: {sum_revenue_B:.2f} руб.")
if sum_revenue_B > sum_revenue_A:
    direction_revenue = "увеличилась"
else:
    direction_revenue = "не увеличилась"
stat_label_revenue = "статистически значимо" if (p_revenue < alpha) else "не статистически значимо"
print(f"    - Выручка в группе B {direction_revenue} по сравнению с A (p = {p_revenue:.4f}, {stat_label_revenue}).\n")

# (c) Как наше изменение повлияло на средний чек заказов на сервисе?
check_A = summary_df.loc[0, 'mean_check_per_order_group']
check_B = summary_df.loc[1, 'mean_check_per_order_group']
print("(c) Средний чек (на уровне группы):")
print(f"    - Группа A: {check_A:.2f} руб, Группа B: {check_B:.2f} руб.")
if check_B > check_A:
    direction_check = "увеличился"
elif check_B < check_A:
    direction_check = "уменьшился"
else:
    direction_check = "не изменился"
stat_label_check = "статистически значимо" if (p_check < alpha) else "не статистически значимо"
print(f"    - Средний чек в группе B {direction_check} по сравнению с A (p = {p_check:.4f}, {stat_label_check}).\n")

# (d) Как наше изменение повлияло на суммарную прибыль сервиса?
sum_profit_A = summary_df.loc[0, 'sum_profit']
sum_profit_B = summary_df.loc[1, 'sum_profit']
print("(d) Суммарная прибыль:")
print(f"    - Группа A: {sum_profit_A:.2f} руб, Группа B: {sum_profit_B:.2f} руб.")
if sum_profit_B > sum_profit_A:
    direction_profit = "увеличилась"
else:
    direction_profit = "уменьшилась"
stat_label_profit = "статистически значимо" if (p_profit < alpha) else "не статистически значимо"
print(f"    - Прибыль в группе B {direction_profit} по сравнению с A (p = {p_profit:.4f}, {stat_label_profit}).")
print(f"    - Инкрементальная прибыль за один дополнительный заказ: {incremental_profit_per_order:.2f} руб.\n")

# (e) Сформулированный вывод: стоит ли использовать акцию на постоянной основе или нет?
print("(e) Рекомендация по дальнейшему использованию акции:")
recommendation = []
# Проверяем сразу три условия:
cond_orders       = (sum_orders_B > sum_orders_A) and (p_orders < alpha)
cond_revenue      = (sum_revenue_B > sum_revenue_A) and (p_revenue < alpha)
cond_incremental  = (incremental_profit_per_order > 0)

# Если все три в "плюсе" — рекомендуем
if cond_orders and cond_revenue and cond_incremental:
    recommendation.append("- В группе B отмечается статистически значимый рост количества заказов и выручки.")
    recommendation.append("- Инкрементальная прибыль на один дополнительный заказ положительна.")
    recommendation.append("- Рекомендуем оставить акцию на постоянной основе: она приносит прибыль и стимулирует объем продаж.")
else:
    recommendation.append("- Результаты неоднозначны:")
    if not cond_orders:
        recommendation.append("    – Не удалось достоверно увеличить число заказов.")
    if not cond_revenue:
        recommendation.append("    – Нет статистически значимого роста суммарной выручки.")
    if not cond_incremental:
        recommendation.append(f"    – Инкрементальная прибыль ({incremental_profit_per_order:.2f} руб.) не положительна.")
    recommendation.append("- В такой ситуации не рекомендуется использовать акцию на постоянной основе.")
print("\n".join(recommendation))


=== Сводка по группам A и B (агрегаты) ===
group  n_users  sum_orders  sum_revenue   sum_profit  mean_orders_per_user  mean_revenue_per_user  mean_profit_per_user  mean_check_per_order_group
    A    50000      149676 2.993472e+08 5.986943e+07               2.99352            5986.943298            1197.38866                 1999.967696
    B    50000      154860 2.942298e+08 5.884595e+07               3.09720            5884.595398            1176.91908                 1899.972684

=== Результаты t-тестов ===
       metric  t_statistic      p_value
     n_orders    11.491051 1.529356e-30
total_revenue    -5.807786 6.349549e-09
average_check  -242.298970 0.000000e+00
 total_profit    -5.807786 6.349549e-09

Разница в количестве заказов (B - A): 5184
Разница в суммарной прибыли  (B - A): -1023479.00 руб.
Инкрементальная прибыль за 1 дополнительный заказ: -197.43 руб.

=== Выводы по вопросам (a)–(e) ===

(a) Количество заказов:
    - Группа A: 149676, Группа B: 154860.
    - Количество з